# Baseline model

In [2]:
import pandas as pd
import numpy as np
import gdown 

## Загрузка данных

In [3]:
# Загрузка тестовых данных
gdown.download(url="https://drive.google.com/uc?export=download&id=1Ud6jFto6e7FW5y0LxxwX8afpmTrvA0u-",  
               output="test_transacrion_df.csv",
               quiet=False)
# Загрузка датафрейма с тестовыми данными
test_df = pd.read_csv("test_transacrion_df.csv", index_col=0)


Downloading...
From: https://drive.google.com/uc?export=download&id=1Ud6jFto6e7FW5y0LxxwX8afpmTrvA0u-
To: /home/noname/projects/hse_mlds_recsys_project/ML/test_transacrion_df.csv
100%|██████████| 52.4M/52.4M [00:06<00:00, 8.32MB/s]


In [4]:
# Загрузка датафрейма с рейтингами
gdown.download(url="https://drive.google.com/uc?export=download&id=1epGrpzB8BEC2t5Od3hrL3x07B1VZIm3c",  
               output="ratings.csv",
               quiet=False)
rating_df = pd.read_csv("ratings.csv", index_col=0)

Downloading...
From: https://drive.google.com/uc?export=download&id=1epGrpzB8BEC2t5Od3hrL3x07B1VZIm3c
To: /home/noname/projects/hse_mlds_recsys_project/ML/ratings.csv
100%|██████████| 211M/211M [00:20<00:00, 10.5MB/s] 


# Метрика качества

In [5]:
def precision_at_k(relevant, predicted, k: int = 10):
    """ 
        Функиця расчета Precision@k
        relevant - релевантные items для одного пользователя
        predicted - рекомендованные items для одного пользователя
    """
    return len(set(relevant[:k]) & set(predicted[:k]))/k 

def rel_item(relevant, predicted):
    """
        Функция рассчитывает количество релевантных Item
    """
    result = [0]*max(len(relevant), len(predicted))
    items = min(len(relevant), len(predicted))
    for i in range(items):
        result[i] = int(relevant[i] == predicted[i])
    return result  


def ap_at_k(relevant, predicted, k: int = 10):
    """ 
        Функция расчета AP@k
        relevant - релевантные items для одного пользователя
        predicted - рекомендованные items для одного пользователя
    """
    y_i = rel_item(relevant=relevant, predicted=predicted)
    p_at_i = [0]*k
    iter_cnt = min(len(relevant), k)
    for i in range(1, iter_cnt+1):
        p_at_i[i-1] = precision_at_k(relevant=relevant, predicted=predicted, k=i)
    return sum([y*p/k for y, p in zip(y_i, p_at_i)])

def map_at_k(relevant, predicted, k: int = 10):
    """ 
        Функция расчета MAP@k
        relevant список по всем пользователям с их релевантными items
        predicted список по всем пользователям с их рекомендованными items
    """
    users = len(relevant)
    sum_apk = 0
    for user in range(users):
        sum_apk += ap_at_k(relevant=relevant[user], predicted=predicted[user], k=k)
    return sum_apk/users

In [6]:
def nap_at_k(relevant, predicted, k: int = 10):
    """ 
        Функция расчета normalize AP@k
        relevant - релевантные items для одного пользователя
        predicted - рекомендованные items для одного пользователя
    """
    y_i = rel_item(relevant=relevant, predicted=predicted)
    p_at_i = [0]*k
    k = min(len(relevant), k)
    for i in range(1, k+1):
        p_at_i[i-1] = precision_at_k(relevant=relevant, predicted=predicted, k=i)
    return sum([y*p/k for y, p in zip(y_i, p_at_i)])

def mnap_at_k(relevant, predicted, k: int = 10):
    """ 
        Функция расчета MAP@k
        relevant список по всем пользователям с их релевантными items
        predicted список по всем пользователям с их рекомендованными items
    """
    users = len(relevant)
    sum_napk = 0
    for user in range(users):
        sum_napk += nap_at_k(relevant=relevant[user], predicted=predicted[user], k=k)
    return sum_napk/users

In [7]:
def hitrate_at_k(relevant, predicted, k: int = 10):
    """
        Функция расчета Hitrate@k
        relevant список по всем пользователям с их релевантными items
        predicted список по всем пользователям с их рекомендованными items
    """
    
    cnt_user = len(predicted) # Количество пользователей    
    cnt_valid_user = 0
    for user in range(cnt_user):
        cnt_valid_user += int(len(set(relevant[user][:k]) & set(predicted[user][:k])) > 0) 

    return cnt_valid_user/cnt_user

In [8]:
def ndsg_at_k(relevant, predicted, k: int = 10):
    """
        Функция расчета nDSG@k
        relevant - релевантные items для одного пользователя
        predicted - рекомендованные items для одного пользователя
    """
    idsg_at_k = sum([1/np.log2(k+1) for k in range(1, k+1)])
    k = min(len(relevant), k)
    dsg_at_k = 0
    for k in range(1, k+1):
        dsg_at_k += int(relevant[k-1] == predicted[k-1])/np.log2(k+1)
    return dsg_at_k/idsg_at_k


## Baseline 

В качестве рекомендации модель возвращает k самых популярных товаров

In [9]:
class Baseline():

    def __init__(self, rating_df: pd.DataFrame):
        self.rating = (rating_df
            .groupby("product_id", as_index=False)["rating"]
            .count()
            .sort_values(by="rating", ascending=False))

    def predict(self, n_recomend: int, k: int = 10):
        """
            Вычисление рекомендаций для n пользователей
        """
        return [self.recommend(k=k)] * n_recomend

    def recommend(self, k: int):
        """
            Расчет рекомендаций
        """
        return np.array(self.rating["product_id"][:k])


In [10]:
# Пользователи с их релевантными покупками
relevant = (test_df.sort_values(by=["user_id", "add_to_cart_order"])
            .groupby(['user_id'])
            .agg({'product_id': 'unique'})['product_id'].to_list())

In [11]:
# Инициализация модели
model = Baseline(rating_df=rating_df)
# Рекомендации
predict = model.predict(n_recomend=len(relevant), k=10)

In [13]:
relevant = relevant[5000:7000]

In [15]:
predict = predict[5000:7000]

Оценки качества Baseline модели

In [16]:
metrics = {}
metrics["MAP@k"] = map_at_k(relevant=relevant, predicted=predict, k=10)
metrics["MNAP@k"] = mnap_at_k(relevant=relevant, predicted=predict, k=10)
metrics["Hitrate@k"] = hitrate_at_k(relevant=relevant, predicted=predict, k=10)
sum_ndsg = 0
sum_precision = 0
for user in range(len(relevant)):
    rel = relevant[user]
    pred = predict[user]
    sum_ndsg += ndsg_at_k(relevant=rel, predicted=pred, k=10)
    sum_precision += precision_at_k(relevant=rel, predicted=pred, k=10)
metrics["AVG_nDSG@k"] = sum_ndsg/len(relevant)
metrics["AVG_Precision@k"] = sum_precision/len(relevant)

In [17]:
metrics

{'MAP@k': 0.005521349206349201,
 'MNAP@k': 0.007864355158730154,
 'Hitrate@k': 0.4375,
 'AVG_nDSG@k': 0.013482733195953554,
 'AVG_Precision@k': 0.061849999999999586}